In [1]:
import glacierml as gl

2022-07-06 18:43:02.086340: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [50]:
df = gl.data_loader()
df=df.drop('Zmed', axis=1)
df.name = 'df'
df

,CenLat,CenLon,Area,Zmin,Zmax,Slope,Aspect,Lmax,Thickness
0,67.91500,18.56400,1.438,1204,1698,16.9,98,1941,72.0
1,67.91000,18.49600,3.696,1207,2070,15.9,282,3954,84.0
2,67.90300,18.56900,3.405,1143,1797,13.2,102,3636,99.0
3,48.35698,-121.05735,2.924,1613,2196,12.8,350,3338,99.0
4,52.17540,-117.28400,16.154,1982,3448,12.6,93,10396,150.0
...,...,...,...,...,...,...,...,...,...
440,-3.05715,37.35070,0.616,5663,5794,9.9,355,905,23.0
441,77.02720,15.48540,0.418,323,587,18.8,144,1084,27.0
442,78.87100,12.16140,5.016,123,684,13.8,350,4111,76.0
443,46.21184,-121.49584,0.027,3396,3529,35.5,307,214,57.0


In [51]:
history_filename, model_filename = gl.build_and_train_model(df)

INFO:tensorflow:Assets written to: saved_models/sm2/sm_10-5/df_1_dnn_MULTI_0.001_0.2_100_0/assets


In [52]:
import tensorflow as tf
model = tf.keras.models.load_model(model_filename)

In [53]:
train_features, test_features, train_labels, test_labels = gl.data_splitter(df)

In [54]:
import numpy as np
y_pred = model.predict(test_features).flatten()
y_true=test_labels
loss = tf.keras.losses.mean_absolute_error(y_true, y_pred)
print(loss)

3/3 [==============================] - 0s 1ms/step
tf.Tensor(31.963617, shape=(), dtype=float32)


In [55]:
import pandas as pd
model_history = pd.read_csv(history_filename)
model_history

,Unnamed: 0,loss,val_loss
0,0,66.072113,59.073658
1,1,65.904251,58.916264
2,2,65.737953,58.757790
3,3,65.570221,58.597706
4,4,65.399429,58.435802
...,...,...,...
95,95,27.450785,23.456352
96,96,27.375595,23.423695
97,97,27.316511,23.381044
98,98,27.253073,23.350662


In [58]:
RGI_All = gl.RGI_loader()
# RGI_All = RGI_All.reset_index()
RGI_All

100%|██████████| 19/19 [00:00<00:00, 30.64it/s]


,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax
0,63.6890,-146.8230,42.0,1936,2385,2725,0.360,346,839
1,63.4040,-146.6680,16.0,1713,2005,2144,0.558,162,1197
2,63.3760,-146.0800,18.0,1609,1868,2182,1.685,175,2106
3,63.3810,-146.1200,19.0,1273,1944,2317,3.681,195,4175
4,63.5510,-147.0570,16.0,1494,1914,2317,2.573,181,2981
...,...,...,...,...,...,...,...,...,...
2747,-53.9860,-37.7325,29.9,310,-999,510,0.042,315,255
2748,-54.8310,-36.1361,23.6,330,-999,830,0.567,200,1130
2749,-54.1884,-37.3018,16.8,10,-999,1110,4.118,308,4329
2750,-68.8656,-90.4266,0.4,170,-999,270,0.011,122,106


In [59]:
RGI = RGI.drop(RGI_All.loc[RGI_All['Zmed']<0].index)
RGI = RGI.drop(RGI.loc[RGI['Lmax']<0].index)
RGI = RGI.drop(RGI.loc[RGI['Slope']<0].index)
RGI = RGI.drop(RGI.loc[RGI['Aspect']<0].index)
RGI = RGI.reset_index()
RGI = RGI.drop('index', axis=1)
RGI = RGI.drop('Zmed', axis=1)
RGI

,CenLat,CenLon,Slope,Zmin,Zmax,Area,Aspect,Lmax
0,61.2240,-144.102,16.0,1239,1719,1.570,281,2183
1,61.2050,-144.491,25.0,1328,1741,0.294,54,815
2,61.1660,-144.833,30.0,872,2367,1.416,294,2380
3,61.2160,-144.369,24.0,1391,1878,0.519,214,1000
4,61.2350,-144.106,24.0,1114,1494,0.214,326,759
...,...,...,...,...,...,...,...,...
131929,-43.4215,170.354,34.7,1231,2098,0.189,116,944
131930,-43.4550,170.349,36.1,1881,2208,0.040,108,331
131931,-43.4400,170.351,39.2,1677,2253,0.184,104,740
131932,-43.4106,170.364,34.0,1627,1928,0.111,135,406


In [60]:
y_pred = model.predict(RGI).flatten()

4123/4123 [==============================] - 3s 607us/step


In [61]:
np.mean(y_pred)

70.40796

In [40]:
print(f'Zmed missing: {len(RGI_All[RGI_All.Zmed<0])}\n'
      f'Slope missing: {len(RGI_All[RGI_All.Slope<0])}\n'
      f'Aspect missing: {len(RGI_All[RGI_All.Aspect<0])}\n'
      f'Lmax missing: {len(RGI_All[RGI_All.Lmax<0])}\n')

Zmed missing: 2779
Slope missing: 13
Aspect missing: 673
Lmax missing: 5482



In [46]:
print(f'Length w/o Zmed: {np.mean(RGI_All[RGI_All.Zmed<0].Lmax)}\n'
      f'Area w/ Zmed: {np.mean(RGI_All[RGI_All.Zmed>0].Area)}\n\n'
      f'Area w/o Slope: {np.mean(RGI_All[RGI_All.Slope<0].Area)}\n'
      f'Area w/ Slope: {np.mean(RGI_All[RGI_All.Slope>0].Area)}\n\n'
      f'Area w/o Aspect: {np.mean(RGI_All[RGI_All.Aspect<0].Area)}\n'
      f'Area w/ Aspect: {np.mean(RGI_All[RGI_All.Aspect>0].Area)}\n\n'
      f'Area w/o Lmax: {np.mean(RGI_All[RGI_All.Lmax<0].Area)}\n'
      f'Area w/ Lmax: {np.mean(RGI_All[RGI_All.Lmax>0].Area)}\n\n')

Length w/o Zmed: 2657.1734436847787
Area w/ Zmed: 2.867712603288387

Area w/o Slope: 0.7229230769230769
Area w/ Slope: 3.454341666396367

Area w/o Aspect: 0.8746745913818721
Area w/ Aspect: 3.459842157586907

Area w/o Lmax: 0.27942429770156835
Area w/ Lmax: 3.528401622602705


